In [ ]:
import os

from customlib.sqldao import DatabaseAccessObject

sql_conn = DatabaseAccessObject(db="MD")

In [ ]:
sql_conn.query_db("show databases")

┌───────────────┐
│ database_name │
│    varchar    │
├───────────────┤
│ my_db         │
│ sample_data   │
└───────────────┘

In [ ]:
sql_conn.query_db(
f"""CREATE SECRET IN MOTHERDUCK (
    TYPE S3,
    KEY_ID '{os.environ['AWS_ACCESS_KEY_ID']}',
    SECRET '{os.environ['AWS_SECRET_ACCESS_KEY']}',
    REGION '{os.environ['REGION']}'
);
"""
)

In [ ]:
sql_conn.query_db("SELECT count(*) FROM 's3://<bucket>/<file>';")